In [1]:
import pandas as pd
import numpy as np

## 1. Introduction 

In the previous notebook, the top five locations for new Lowe's and Home Depot stores were calculated based upon logistic regression models and using census data.  In that analysis, we had no information about the real estate or any consideration about how "hot" or trendy an area might be, so our data only showed the "best" locations as they currently stood.  In this analysis, instead of using static data, we will take into consideration the "hotness" or a market and incorporate this information into the calculation. The real estate data is taken from August 2018, so it is not currently as of 2021 up-to-date, but provides useful analysis nonetheless. 

## 2. Configure the Datasets

In [2]:
# Load the state region data 
state_region = pd.read_csv('data/state_region.csv')
state_region.head()

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [3]:
# Check shape
state_region.shape

(51, 4)

The columns `State Code` and `Region` are redundant since we just want the state and general region, and since `Division` provides a more specific general region, we will drop `State Code` and `Region` and keep the other two: 

In [4]:
# Drop State Code and Region 
state_region.drop(['State', 'Division'], axis=1, inplace=True)

Let's look now at the real estate data: 

In [5]:
# Load the real estate data
real_estate = pd.read_csv('data/RDC_MarketHotness_Monthly.csv')
real_estate.head()

,CountyFIPS,Month,CountyName,ZipName,Nielsen HH Rank,Hotness Rank Within County,Hotness Rank Within CBSA,Hotness Rank,Hotness Rank M/M,Hotness Rank Y/Y,...,Views Per Property (vs US),Views Per Property (vs CBSA),Views Per Property (vs County),Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Median Listing Price (vs US),Median Listing Price (vs CBSA),Median Listing Price (vs County),Quality Flag
0,25009,201807,Essex,"Danvers, MA",3564,2.0,4.0,48.0,51.0,521.0,...,10.8310,5.0261,3.5602,475000.0,0.0349,0.0820,1.5886,0.9136,0.9542,NaN
1,25023,201807,Plymouth,"Rockland, MA",6660,3.0,33.0,253.0,-190.0,-33.0,...,8.2535,3.8301,4.2774,330000.0,0.0124,0.0963,1.1037,0.6347,0.7333,NaN
2,25005,201807,Bristol,"Norton, MA",6986,3.0,3.0,615.0,-178.0,-134.0,...,7.6901,5.3010,4.1364,399900.0,0.0321,0.0811,1.3375,1.1429,1.0837,NaN
3,25017,201807,Middlesex,"North Chelmsford, MA",9365,6.0,12.0,89.0,-29.0,-74.0,...,7.5493,3.5033,3.1716,302500.0,0.0003,0.0807,1.0117,0.5818,0.4879,NaN
4,25017,201807,Middlesex,"Lowell, MA",4143,21.0,42.0,312.0,-95.0,45.0,...,6.6197,3.0719,2.7811,291500.0,0.0055,0.0053,0.9749,0.5607,0.4702,NaN


In [6]:
# Check tail of data
real_estate.tail()

,CountyFIPS,Month,CountyName,ZipName,Nielsen HH Rank,Hotness Rank Within County,Hotness Rank Within CBSA,Hotness Rank,Hotness Rank M/M,Hotness Rank Y/Y,...,Views Per Property (vs US),Views Per Property (vs CBSA),Views Per Property (vs County),Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Median Listing Price (vs US),Median Listing Price (vs CBSA),Median Listing Price (vs County),Quality Flag
16736,48141,201807,El Paso,"San Elizario, TX",9233,22.0,22.0,16723.0,-298.0,118.0,...,0.0704,0.1250,0.1250,38500.0,0.0000,0.0000,0.1288,0.2239,0.2239,NaN
16737,18107,201807,Montgomery,"Waveland, IN",21751,2.0,2.0,16740.0,-228.0,-8193.0,...,0.0704,0.0704,0.0704,404990.0,0.0062,0.0000,1.3545,2.7017,2.7017,NaN
16738,2198,201807,Prince of Wales Hyder,"Klawock, AK",23487,1.0,13.0,16581.0,NaN,NaN,...,0.0423,NaN,0.5000,350000.0,-0.0305,0.0145,1.1706,NaN,1.2111,NaN
16739,46011,201807,Brookings,"Aurora, SD",22729,2.0,2.0,14725.0,-2698.0,NaN,...,0.0282,0.0833,0.0833,191400.0,0.0079,NaN,0.6401,0.9119,0.9119,NaN
16740,1,201807,United States,United States,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,299000.0,0.0000,0.0873,NaN,NaN,NaN,NaN


It looks like there is one row that is in the dataset that is for the entire United States, which we clearly do not want, so let's get rid of that.  Let's also create a new column to extract the state abbreviation from `ZipName` so that we can use this to merge with the other dataset: 

In [7]:
# Remove CountyName = 'United States'
real_estate = real_estate[real_estate.CountyName != 'United States']

# Extract the state abbreviation from ZipName and create new column
import re
real_estate['State Code'] = [re.search('\w+, ([A-Z]{2})', x).group(1) for x in real_estate.ZipName]

Let's check the dataset again: 

In [8]:
# Check dataset
real_estate.sample(5)

,CountyFIPS,Month,CountyName,ZipName,Nielsen HH Rank,Hotness Rank Within County,Hotness Rank Within CBSA,Hotness Rank,Hotness Rank M/M,Hotness Rank Y/Y,...,Views Per Property (vs CBSA),Views Per Property (vs County),Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Median Listing Price (vs US),Median Listing Price (vs CBSA),Median Listing Price (vs County),Quality Flag,State Code
470,25017,201807,Middlesex,"Woburn, MA",1290,26.0,55.0,470.0,-344.0,-354.0,...,1.0980,0.9941,545000.0,0.0188,0.2114,1.8227,1.0483,0.8790,NaN,MA
9473,17201,201807,Winnebago,"Rockford, IL",6534,14.0,16.0,7420.0,-5060.0,3964.0,...,0.5088,0.5088,55750.0,-0.0388,0.2133,0.1865,0.3846,0.4256,NaN,IL
11214,36045,201807,Jefferson,"Alexandria Bay, NY",18051,6.0,6.0,13401.0,1241.0,1861.0,...,1.8846,1.8846,223000.0,0.3618,-0.2504,0.7458,1.3125,1.3125,NaN,NY
6256,56025,201807,Natrona,"Casper, WY",3265,1.0,1.0,5316.0,976.0,1400.0,...,1.3103,1.3103,217750.0,-0.0532,0.0627,0.7283,0.8127,0.8127,NaN,WY
15677,21035,201807,Calloway,"New Concord, KY",21113,2.0,2.0,16295.0,-189.0,-164.0,...,0.6486,0.6486,389000.0,-0.0019,-0.0202,1.3010,2.1623,2.1623,NaN,KY


Everything looks ok.  So now let's merge the real estate dataset with the state information: 

In [9]:
# Left join state_region to real estate datasets on 'StateCode'
real_estate = pd.merge(real_estate, state_region, how='left', on='State Code', copy=False)
real_estate.sample(5)

,CountyFIPS,Month,CountyName,ZipName,Nielsen HH Rank,Hotness Rank Within County,Hotness Rank Within CBSA,Hotness Rank,Hotness Rank M/M,Hotness Rank Y/Y,...,Views Per Property (vs County),Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Median Listing Price (vs US),Median Listing Price (vs CBSA),Median Listing Price (vs County),Quality Flag,State Code,Region
1459,25005,201807,Bristol,"North Attleboro, MA",4175,7.0,8.0,1420.0,-206.0,94.0,...,0.9848,359900.0,0.0069,0.0299,1.2037,1.0286,0.9753,NaN,MA,Northeast
2664,6017,201807,El Dorado,"Rescue, CA",13816,7.0,63.0,3281.0,-160.0,-645.0,...,0.9820,654900.0,-0.0368,0.1243,2.1903,1.4268,1.2427,NaN,CA,West
4726,36119,201807,Westchester,"Bronxville, NY",4537,22.0,325.0,7874.0,-390.0,1926.0,...,1.2609,749000.0,-0.1019,0.3618,2.5050,1.4000,1.0700,NaN,NY,Northeast
15412,5129,201807,Searcy,"Marshall, AR",14028,2.0,35.0,15911.0,-509.0,851.0,...,0.8966,85000.0,0.0000,-0.2911,0.2843,NaN,0.9444,NaN,AR,South
284,33011,201807,Hillsborough,"Merrimack, NH",4630,1.0,1.0,184.0,564.0,547.0,...,1.4453,311000.0,0.0079,-0.0278,1.0401,0.9569,0.9569,NaN,NH,Northeast


As one final check, let's go ahead and drop the duplicate FIPS values and keep the non-duplicated values.  Many of these rows are repeats, but we only want a single row per FIPS value.  We will also reset the index: 

In [10]:
# Drop FIPS duplicates 
real_estate.drop_duplicates(subset='CountyFIPS', inplace=True)

# Reset the index
real_estate.reset_index(drop=True, inplace=True)
real_estate.head()

,CountyFIPS,Month,CountyName,ZipName,Nielsen HH Rank,Hotness Rank Within County,Hotness Rank Within CBSA,Hotness Rank,Hotness Rank M/M,Hotness Rank Y/Y,...,Views Per Property (vs County),Median Listing Price,Median Listing Price M/M,Median Listing Price Y/Y,Median Listing Price (vs US),Median Listing Price (vs CBSA),Median Listing Price (vs County),Quality Flag,State Code,Region
0,25009,201807,Essex,"Danvers, MA",3564,2.0,4.0,48.0,51.0,521.0,...,3.5602,475000.0,0.0349,0.0820,1.5886,0.9136,0.9542,NaN,MA,Northeast
1,25023,201807,Plymouth,"Rockland, MA",6660,3.0,33.0,253.0,-190.0,-33.0,...,4.2774,330000.0,0.0124,0.0963,1.1037,0.6347,0.7333,NaN,MA,Northeast
2,25005,201807,Bristol,"Norton, MA",6986,3.0,3.0,615.0,-178.0,-134.0,...,4.1364,399900.0,0.0321,0.0811,1.3375,1.1429,1.0837,NaN,MA,Northeast
3,25017,201807,Middlesex,"North Chelmsford, MA",9365,6.0,12.0,89.0,-29.0,-74.0,...,3.1716,302500.0,0.0003,0.0807,1.0117,0.5818,0.4879,NaN,MA,Northeast
4,42071,201807,Lancaster,"Strasburg, PA",12705,5.0,5.0,2715.0,4216.0,-2698.0,...,3.5364,264900.0,0.0392,0.0236,0.8860,0.9636,0.9636,NaN,PA,Northeast


In [11]:
# Check final shape
real_estate.shape

(2669, 35)

In [12]:
# Check final columns 
real_estate.columns

Index(['CountyFIPS', 'Month', 'CountyName', 'ZipName', 'Nielsen HH Rank',
       'Hotness Rank Within County', 'Hotness Rank Within CBSA',
       'Hotness Rank ', 'Hotness Rank M/M', 'Hotness Rank Y/Y',
       'Hotness Score', 'Supply Score', 'Demand Score', 'Median DOM',
       'Median DOM M/M', 'Median DOM M/M Perc', 'Median DOM Y/Y',
       'Median DOM Y/Y Perc', 'Median DOM (vs US)', 'Median DOM (vs CBSA)',
       'Median DOM (vs County)', 'Views Per Property M/M',
       'Views Per Property Y/Y', 'Views Per Property  (vs US)',
       'Views Per Property  (vs CBSA)', 'Views Per Property  (vs County)',
       'Median Listing Price', 'Median Listing Price M/M',
       'Median Listing Price Y/Y', 'Median Listing Price  (vs US)',
       'Median Listing Price  (vs CBSA)', 'Median Listing Price  (vs County)',
       'Quality Flag', 'State Code', 'Region'],
      dtype='object')

Great.  The final dataset now looks ok and we can proceed.

## 3. EDA 

To get a sense of the real estate data, let's explore it a bit.  Since we are interested in where the greatest current demand for housing is in the U.S., let's look at `Demand Score`: 

In [13]:
# Check demand score by regions of the country 
real_estate.groupby('Region')['Demand Score'].mean().sort_values(ascending=False)

Region
Northeast    72.936047
Midwest      52.505150
West         51.710213
South        44.266387
Name: Demand Score, dtype: float64

In [14]:
# Check demand score by state 
real_estate.groupby('State Code')['Demand Score'].mean().sort_values(ascending=False)[:5]

State Code
MA    94.456061
CT    90.195053
NH    87.770476
RI    87.760320
NJ    86.581095
Name: Demand Score, dtype: float64

Let's take a look at the top metro areas by best demand score: 

In [17]:
# Using the FIPS codes, join the POP_2010 column from hdl and real_estate
hdl = pd.read_csv('data/hdl.csv')
hdl.drop([1654,2922,2950], inplace=True)
means = hdl.mean(axis=0)
hdl.fillna(means, inplace=True)

pop_data = hdl[['county', 'pop_2010']]
realtor_withpop = pd.merge(real_estate, pop_data, how='inner', \
                           left_on='CountyFIPS', right_on='county')

# Filter by pop_2010 > 1mil, then find highest "Demand Score"
realtor_withpop[realtor_withpop.pop_2010 > 1e6][['CountyName', 'State Code','Demand Score']]\
.sort_values('Demand Score', ascending=False)

,CountyName,State Code,Demand Score
3,Middlesex,MA,99.982078
6,Los Angeles,CA,99.922337
16,Bexar,TX,99.755063
17,Wayne,MI,99.743115
18,Philadelphia,PA,99.737141
22,Tarrant,TX,99.623633
23,Franklin,OH,99.557919
24,Dallas,TX,99.545970
27,San Diego,CA,99.510126
28,Cuyahoga,OH,99.504152


So we see that Middlesex, MA has the best demand score by metro area.  Checking the demand scores with what we saw on the original map, it appears that the Northeast, Texas, Ohio, california, and Florida are all good regions for new stores.  This agrees somewhat with our model; we have many predictions in Californi and the Northeast.  Let us now take the following columns and incorporate them into our model and see if our new predictions are better than in the previous notebook: 

In [19]:
# Subset the realtor_data to only contain the columns we want to merge with the hdl_data
merge_to = real_estate[['CountyFIPS', 'Median Listing Price', 'Demand Score', 'Hotness Score', 'Nielsen HH Rank']]

# Rename columns
demographic = hdl.filter(regex='_').copy()
demo_features = demographic.drop('pctwhite_2000', axis=1)
merge_to.columns = ['county', 'Median.Listing.Price', 'Demand.Score', 'Hotness.Score', 'Nieleson.HH.Rank']
X = demo_features.copy()
X['county'] = hdl.county

# Merge this data with hdl_data on county
merged = pd.merge(X, merge_to, how='left', on='county', copy=False)
merged.drop('county',axis=1, inplace=True)
merged.fillna(merged.mean(axis=0), axis=0, inplace=True)

In [24]:
# Copy data into training and testing sets, drop earlier added probability columns along with other irrelevant columns
hdl['HDexists'] = hdl.HDcount>0
hdl['Lexists'] = hdl.Lcount>0 
interm = pd.merge(X, hdl, how='left', on='county')
hd_y = interm.HDexists

interm2 = pd.merge(X, hdl, how='left', on='county')
l_y = interm2.Lexists


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
ss = StandardScaler()
X_standard = ss.fit_transform(merged)


# Split the data into train and test portions to test accuracy
hd_X_train, hd_X_test, hd_y_train, hd_y_test = train_test_split(X_standard, hd_y, random_state=42)
l_X_train, l_X_test, l_y_train, l_y_test = train_test_split(X_standard, l_y, random_state=42)

In [25]:
# Train the models
from sklearn.linear_model import LogisticRegressionCV
pen = 'l1'

hd_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', penalty=pen, solver='liblinear', \
                                max_iter=3000, random_state=42).fit(hd_X_train, hd_y_train)
l_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', penalty=pen, solver='liblinear', \
                               max_iter=3000, random_state=42).fit(l_X_train, l_y_train)

In [26]:
# Home Depot store predicition accuracy
hd_logit.score(hd_X_test, hd_y_test)

0.8982188295165394

In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(hd_y_test, hd_logit.predict(hd_X_test))

array([[544,  42],
       [ 38, 162]])

In [29]:
# Lowes store predicition accuracy
l_logit.score(l_X_test, l_y_test)

0.8905852417302799

In [30]:
confusion_matrix(l_y_test, l_logit.predict(l_X_test))

array([[507,  45],
       [ 41, 193]])

In [31]:
# Train on the full data
pen = 'l2'

hd_full_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', penalty=pen, solver='liblinear', \
                                     max_iter=1000, random_state=42).fit(X_standard, hd_y)
l_full_logit = LogisticRegressionCV(Cs=10, cv=5, class_weight='balanced', penalty=pen, solver='liblinear', \
                                    max_iter=1000, random_state=42).fit(X_standard, l_y)

In [32]:
area_info = hdl[['areaname','state', 'county']].copy()
area_info.index = range(0,3143)
area_info['hd_store_prob'] = hd_full_logit.predict_proba(X_standard)[:,1]
area_info['l_store_prob']  = l_full_logit.predict_proba(X_standard)[:,1]

In [33]:
# Create new column that is the sum of the two probability columns for sorting purposes
area_info['prob_sum'] = area_info.hd_store_prob + area_info.l_store_prob

In [34]:
# For Tool Time, show only locations where there are 1 or no stores for HD and Lowes,
# sort by prob_sum descending and show top 5 areas predicted to be the best store locations
area_info[(interm.Lcount <= 1) & (interm2.HDcount <= 1)].sort_values(by='prob_sum', ascending=False).head(10)

,areaname,state,county,hd_store_prob,l_store_prob,prob_sum
1375,Ramsey,MN,27123,1.000000,0.999978,1.999978
107,Pinal,AZ,4021,0.999647,1.000000,1.999647
1263,Ingham,MI,26065,0.999884,0.999705,1.999589
319,District of Columbia,DC,11001,1.000000,0.999127,1.999127
2762,Webb,TX,48479,0.999190,0.999858,1.999048
306,Weld,CO,8123,0.999001,0.999928,1.998929
1300,Ottawa,MI,26139,0.999824,0.998831,1.998655
2992,Yakima,WA,53077,0.998883,0.998576,1.997459
2222,Jackson,OR,41029,0.997995,0.998418,1.996413
2278,Lackawanna,PA,42069,0.998267,0.997768,1.996036


After incorporating the new data, we see that the top five counties basically remain the same.  Based upon the new scores, after we use the l2 ridge penalty, new features seem to have minimal impact on the accuracy of the models.  For the lasso penalty, new features seem to improve slightly.  In this case, a reasonable conclusion is that adding new features does not seem to add much value to the model, therefore we likely prefer our previous model.  

As an alternative to using Census and demographic data, a builder may want to consider something like commercial real estate data, because there would likely be a large demand for tool supplies in an area with a large amount of commerical real estate development.  Another option would be to incorporate data about where new neighborhoods may be built.  If the number of new developments is large enough, it may make building a tool store in the region highly profitable in comparison to some of the predictions we obtained using our model data alone. 